In [1]:
!pip install -q kaggle

In [2]:
!mkdir .kaggle

In [3]:
!pip install opendatasets

In [4]:
import opendatasets as od

In [5]:
od.download("https://www.kaggle.com/datasets/smeschke/four-shapes")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: yunchanho
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/smeschke/four-shapes


100%|██████████| 21.6M/21.6M [00:00<00:00, 89.5MB/s]


In [7]:
import numpy as np
import pandas as pd
import os
import glob
import cv2
from sklearn.utils import shuffle
import keras
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout
from tensorflow.keras.layers import BatchNormalization

In [9]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout
from tensorflow.keras.layers import BatchNormalization

In [10]:
#Define a path to our data
circle_train_path = 'four-shapes/shapes/*circle'
square_train_path = 'four-shapes/shapes/*square'
star_train_path = 'four-shapes/shapes/*star'
triangle_train_path = 'four-shapes/shapes/*triangle'
cir_path = os.path.join(circle_train_path,'*g')
sq_path = os.path.join(square_train_path,'*g')
str_path = os.path.join(star_train_path,'*g')
trg_path = os.path.join(triangle_train_path,'*g')
#create train array
x_train = []
y_train = []

In [12]:
print("Reading Training Data")
#reading data to memory
file_1 = glob.glob(cir_path)
for f1 in file_1:
  img = cv2.imread(f1)
  x_train.append(img)
  y_train.append([1,0,0,0])
print("25% Complete")
file_2 = glob.glob(sq_path)
for f2 in file_2:
  img = cv2.imread(f2)
  x_train.append(img)
  y_train.append([0,1,0,0])
print("50% Complete")
file_3 = glob.glob(str_path)
for f3 in file_3:
  img = cv2.imread(f3)
  x_train.append(img)
  y_train.append([0,0,1,0])
print("75% Complete")
file_4 = glob.glob(trg_path)
for f4 in file_4:
  img = cv2.imread(f4)
  x_train.append(img)
  y_train.append([0,0,0,1])

x_train = np.array(x_train)
y_train = np.array(y_train)

x_train, y_train = shuffle(x_train,y_train)
print("Finished Reading Training Data")

Reading Training Data
25% Complete
50% Complete
75% Complete
Finished Reading Training Data


In [13]:
#create a model
model = Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape = (200,200,3)))
model.add(MaxPooling2D((2,2),strides = (2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64,(2,2),activation='relu'))
model.add(MaxPooling2D((2,2),strides = (2,2)))
model.add(Conv2D(128,(2,2),activation='relu'))
model.add(MaxPooling2D((2,2),strides = (2,2)))
model.add(BatchNormalization())
model.add(Conv2D(256,(5,5),activation='relu'))
model.add(MaxPooling2D((2,2),strides = (2,2)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(4096,activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(4,activation='softmax'))

In [ ]:
#model compile, fit and save
model.compile(loss=keras.losses.categorical_crossentropy,
optimizer=keras.optimizers.SGD(lr=0.00001),
metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size = 1,epochs = 2,verbose = 1,validation_split = 0.0)
model.save('shape_recg.h5')
#del model

Epoch 1/2
 7042/14970 [=============>................] - ETA: 1:24:36 - loss: nan - accuracy: 0.2484

In [ ]:
#load our model again
from keras.models import load_model
model = load_model('shape_recg.h5')

In [ ]:
import cv2
import numpy as np
#test model on image
dir_path = 'four-shapes/shapes/circle/2.png'
test_img = cv2.imread(dir_path)
test_img = test_img.reshape(1,200,200,3)
Y = model.predict(test_img)[0]
val = np.argmax(Y)
if(val == 0):
  print("Circle")
elif(val == 1):
  print("Square")
elif(val == 2):
  print("Star")
else:
  print("Triangle")